In [ ]:
%pip install yfinance
%pip install pandas
%pip install numpy

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime


ticker = "AAPL"
start_date = "2023-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")


try:
    data = yf.download(
        tickers=ticker,
        start=start_date,
        end=end_date,
        progress=True  # 显示下载进度条
    )

    # 确保数据不为空
    if data.empty:
        raise ValueError("没有获取到数据，请检查股票代码和日期范围")

except Exception as e:
    print(f"下载数据时出错: {e}")
    exit()


data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data['SMA_50'] = data['Close'].rolling(window=50, min_periods=1).mean()

print(data)


[*********************100%***********************]  1 of 1 completed

Price             Open        High         Low       Close     Volume  \
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL   
Date                                                                    
2023-01-03  128.613993  129.226060  122.582127  123.470619  112117500   
2023-01-04  125.267347  127.014716  123.480495  124.744125   89113600   
2023-01-05  125.504275  126.136090  123.164587  123.421257   80962700   
2023-01-06  124.398611  128.623871  123.292931  127.962440   87754700   
2023-01-09  128.801572  131.703978  128.228987  128.485657   70790800   
...                ...         ...         ...         ...        ...   
2025-05-22  200.710007  202.750000  199.699997  201.360001   46742400   
2025-05-23  193.669998  197.699997  193.460007  195.270004   78432900   
2025-05-27  198.300003  200.740005  197.429993  200.210007   56288500   
2025-05-28  200.589996  202.729996  199.899994  200.419998   45339700   
2025-05-29  203.580002  203.809998  198.509995  199

In [ ]:
plt.figure(figsize=(14, 8), dpi=100)
sns.set_style("whitegrid")

# Price and Deals
ax1 = plt.subplot(2, 1, 1)
ax2 = plt.subplot(2, 1, 2)


ax1.plot(data.index, data['Close'],
         label='Closing Price',
         color='#1f77b4',
         linewidth=1.5,
         alpha=0.7)


ax1.plot(data.index, data['SMA_50'],
         label='50-Day SMA',
         color='#ff7f0e',
         linestyle='--',
         linewidth=1.5)


ax1.fill_between(data.index,
                data['Low'].values,
                data['High'].values,
                color='skyblue',
                alpha=0.2,
                label='Daily Range')


colors = ['red' if close < open else 'green'
          for open, close in zip(data['Open'], data['Close'])]

ax2.bar(data.index,
        data['Volume'],
        color=colors,
        alpha=0.7,
        width=1)


ax1.set_title(f'{ticker} Stock Price: {start_date} to {end_date}',
              fontsize=16,
              fontweight='bold')
ax1.set_ylabel('Price (USD)', fontsize=12)
ax1.legend(loc='upper left', frameon=True)
ax1.grid(True, linestyle='--', alpha=0.7)


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))


ax2.set_title('Trading Volume', fontsize=14)
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, linestyle='--', alpha=0.3)


plt.gcf().autofmt_xdate()


company = yf.Ticker(ticker)
info = company.info
plt.figtext(0.15, 0.01,
            f"Company: {info.get('longName', 'N/A')} | Sector: {info.get('sector', 'N/A')} | Market Cap: ${info.get('marketCap', 0)/1e9:.2f}B",
            fontsize=10,
            ha='left')


plt.tight_layout()
plt.show()